In [1]:
import os

db_dir = os.environ["DATA"] + "PatImgXAI_data/dblocaltest0.1/"
test_dataset_filename = "rowcircles_test.csv"

model_dir = os.environ["DATA"] + "models/db_vlocaltest0.1/rowcircles/"
bad_model_dir = os.environ["DATA"] + "models/db_vlocaltest0.1/rowcircles_bad/"



In [2]:
from xaipatimg.datagen.dbimg import load_db

db = load_db(db_dir)

In [3]:
from xaipatimg.ml.xai import generate_cam_resnet18

# generate_cam_resnet18("gradcam", db_dir, test_dataset_filename, model_dir, "cuda:0")

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from xaipatimg.ml.xai import generate_shap_resnet18

# generate_shap_resnet18(db_dir, test_dataset_filename, model_dir, "cuda:0", n_jobs=20, dataset_size=500)

In [5]:
import numpy as np
from xaipatimg.ml.xai import generate_shap_resnet18

# generate_shap_resnet18(db_dir, test_dataset_filename, model_dir, "cuda:0", n_jobs=20, dataset_size=500, masker="ndarray")


Counterfactuals

In [6]:
import numpy as np

def extract_rows_with_only_circles(img_content):
    circles_counter = np.zeros(6,)
    non_circles_counter = np.zeros(6,)

    for c in img_content:
        if c["shape"] == "circle":
            circles_counter[c["pos"][1]] += 1
        else:
            non_circles_counter[c["pos"][1]] += 1

    return np.logical_and(circles_counter >= 1, non_circles_counter == 0)

def exist_row_with_only_circles(img_content):
    return np.sum(extract_rows_with_only_circles(img_content)) >= 1

In [7]:
print(exist_row_with_only_circles(db["fe64ac78688b11f0908cd4d8534cb0f8"]["content"]))

print(exist_row_with_only_circles(db["f61c7762688b11f0908cd4d8534cb0f8"]["content"]))


True
False


In [12]:
db["fe64ac78688b11f0908cd4d8534cb0f8"]["content"]

[{'shape': 'square', 'pos': [0, 1], 'color': '#87C09C'},
 {'shape': 'triangle', 'pos': [0, 2], 'color': '#7AB0CD'},
 {'shape': 'triangle', 'pos': [1, 0], 'color': '#F86C62'},
 {'shape': 'triangle', 'pos': [1, 1], 'color': '#7AB0CD'},
 {'shape': 'circle', 'pos': [1, 4], 'color': '#F86C62'},
 {'shape': 'circle', 'pos': [2, 0], 'color': '#7AB0CD'},
 {'shape': 'triangle', 'pos': [2, 1], 'color': '#7AB0CD'},
 {'shape': 'triangle', 'pos': [2, 2], 'color': '#87C09C'},
 {'shape': 'triangle', 'pos': [3, 1], 'color': '#F86C62'},
 {'shape': 'square', 'pos': [3, 4], 'color': '#87C09C'},
 {'shape': 'circle', 'pos': [4, 1], 'color': '#F4D67B'},
 {'shape': 'square', 'pos': [4, 4], 'color': '#F86C62'},
 {'shape': 'circle', 'pos': [4, 5], 'color': '#F86C62'},
 {'shape': 'circle', 'pos': [5, 2], 'color': '#87C09C'},
 {'shape': 'square', 'pos': [5, 3], 'color': '#87C09C'},
 {'shape': 'triangle', 'pos': [5, 4], 'color': '#F4D67B'}]

In [8]:
from xaipatimg.datagen.utils import gen_rand_sym, PatImgObj
import numpy as np
COLORS  = ["#F86C62", "#7AB0CD", "#F4D67B", "#87C09C"]

def rowcircles_counterfactuals(img_entry, is_pos, nb_cf):

    output_cf = []

    # Case we are searching for negative counterfactuals
    if is_pos:
        # Extracting indices of lines with only circles
        rows_with_only_circles = np.nonzero(extract_rows_with_only_circles(img_entry["content"]))[0]

        # Iterating over all counterfactuals to generate.
        for _ in range(nb_cf):

            patimgobj = PatImgObj(img_entry)

            # Randomly setting a non circle symbol in every row that only contains circles
            for j_row in rows_with_only_circles:
                patimgobj.set_symbol(posx=np.random.choice(np.arange(img_entry["division"][0])),
                                     posy=j_row,
                                     value=gen_rand_sym(shapes=["square", "triangle"], colors=COLORS))

            output_cf.append(patimgobj.get_img_dict())

    # Case we are looking for a positive counterfactual
    else:
        patimgobj = PatImgObj(img_entry)

        # Creating the list of lines which are not empty in a random order
        non_empty_lines_rnd = np.setdiff1d(np.arange(img_entry["division"][1]), patimgobj.get_empty_lines())
        np.random.shuffle(non_empty_lines_rnd)

        for idx in range(min(nb_cf, len(non_empty_lines_rnd))):
            patimgobj = PatImgObj(img_entry)
            patimgobj.change_shapes_of_line(non_empty_lines_rnd[idx], "circle")
            output_cf.append(patimgobj.get_img_dict())

    return output_cf


In [9]:
from xaipatimg.ml.xai import generate_counterfactuals_resnet18

generate_counterfactuals_resnet18(db_dir, test_dataset_filename, model_dir, rowcircles_counterfactuals, 5, None, 1, None)

Loading dataset content for rowcircles_test.csv


100%|██████████| 200/200 [00:01<00:00, 161.14it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Generating counterfactual images


  0%|          | 0/200 [00:00<?, ?it/s]

Loading dataset content for /tmp/tmpz5mj85uw/dataset.csv


...


100%|██████████| 5/5 [00:00<00:00, 217.53it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
 99%|█████████▉| 198/200 [01:03<00:00,  3.20it/s]

Loading dataset content for /tmp/tmpzga4tgp_/dataset.csv



100%|██████████| 5/5 [00:00<00:00, 187.95it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|█████████▉| 199/200 [01:03<00:00,  3.19it/s]

Loading dataset content for /tmp/tmpajo2338a/dataset.csv



100%|██████████| 5/5 [00:00<00:00, 187.15it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|██████████| 200/200 [01:03<00:00,  3.13it/s]


In [10]:
from xaipatimg.ml.xai import generate_counterfactuals_resnet18

generate_counterfactuals_resnet18(db_dir, test_dataset_filename, bad_model_dir, rowcircles_counterfactuals, 5, None, 1, None)

Loading dataset content for rowcircles_test.csv


100%|██████████| 200/200 [00:01<00:00, 157.51it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Generating counterfactual images


  0%|          | 0/200 [00:00<?, ?it/s]

Loading dataset content for /tmp/tmp_o0cmqu2/dataset.csv


...


100%|██████████| 5/5 [00:00<00:00, 173.59it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
 99%|█████████▉| 198/200 [01:01<00:00,  3.36it/s]

Loading dataset content for /tmp/tmps42y3arr/dataset.csv



100%|██████████| 5/5 [00:00<00:00, 225.07it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|█████████▉| 199/200 [01:02<00:00,  3.38it/s]

Loading dataset content for /tmp/tmp1umokp_h/dataset.csv



100%|██████████| 5/5 [00:00<00:00, 217.20it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|██████████| 200/200 [01:02<00:00,  3.20it/s]
